<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Lucas Carrasco
- Nombre de alumno 2: Nicolás Herrera


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/vspartamo/MDS7202)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Jueves a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
#!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
# Inserte su código aquí
df_retail = pd.read_pickle('online_retail_data.pickle')
df_retail.head()

### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [4]:
import plotly.express as px

def explore_data(dataframe_in):
    numeric_cols = dataframe_in.select_dtypes(include=[np.number]).columns
    
    for col in numeric_cols:
        fig = px.histogram(dataframe_in, x=col, title=f"Histograma de {col}")
        fig.show()

    nulos = dataframe_in.isnull().sum()
    print("Conteo de datos nulos por variable:")
    print(nulos)


In [ ]:
explore_data(df_retail)

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).
>**Nota: No modificar el método set_output de la clase IQR**

**Respuesta:**

In [6]:
class IQR(BaseEstimator, TransformerMixin):

  def __init__(self, lambda_): #lambda es un string reservado en python, por lo que se agrega un _
    self.lambda_ = lambda_
  

  def fit(self, X):
    self.q1 = X.quantile(0.25, numeric_only= "True")
    self.q3 = X.quantile(0.75, numeric_only= "True")
    self.ric = self.q3 - self.q1
    return self

  def transform(self, X):
    limite_superior = self.q3 + self.lambda_*self.ric
    limite_inferior = self.q1 - self.lambda_*self.ric

    X_cortado = X.clip(lower = limite_inferior, upper = limite_superior, axis = 1)

    return X_cortado

  def set_output(self,transform='default'):
    #No modificar esta función
    return self

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.
>**Nota: Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.**

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Definicion las variables que pasarán por cada pipeline
#numerical_columns = df_retail.select_dtypes('float' or 'int').columns
#categorical_columns = df_retail.select_dtypes('object').columns

# Definir las variables numéricas y categóricas
numerical_columns = ['Price', 'Quantity']
categorical_columns = ['Invoice', 'StockCode', 'Description', 'InvoiceDate', 'Country', 'Customer ID']
lambda_ = 1.5

numeric_transformations = Pipeline([
    ('remove_outliers', IQR(lambda_))
])

column_transformer = ColumnTransformer([('numerical', numeric_transformations, numerical_columns),
                                        ('categorical', 'passthrough', categorical_columns)
                                        ],
                                        verbose_feature_names_out = False)

column_transformer.set_output(transform='pandas')

df_iqr = column_transformer.fit_transform(df_retail)

explore_data(df_retail)
explore_data(df_iqr)

*Reporte los cambios observados aquí.*
```
Al aplicar el IQR se cortan los valores extremos, lo que hace que los datos restantes estén todos en un rango más pequeño. De esta forma, el gráfico también se ve mucho mejor, ya que inicialmente el rango de los datos era muy amplio por los outliers, por lo que no se podía apreciar bien la distribución real de los datos.

Al aumentar el valor de lambda, se le otorga un rango más amplio a los datos, por lo que se cortan menos valores. Esto hace que existan datos más extremos, y para valores de lambda muy altos se dejan de eliminar los outliers.

```

### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [1 punto]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

>**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [8]:
from sklearn.impute import SimpleImputer

categoric_transformations = Pipeline([
    ('mode_imputer', SimpleImputer(strategy='most_frequent'))
])


numeric_transformations = Pipeline([
    ('remove_outliers', IQR(lambda_=1.5)), 
    ('mean_imputer', SimpleImputer(strategy='mean')) 
])

column_transformer_updated = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformations, numerical_columns),
        ('categorical', categoric_transformations, categorical_columns)
    ],
    verbose_feature_names_out=False
)

column_transformer_updated.set_output(transform='pandas')

df_mean_imputer = column_transformer_updated.fit_transform(df_retail)

In [ ]:
print("Distribución con imputación por media:")
explore_data(df_mean_imputer)

print("Distribución después de aplicar IQR:")
explore_data(df_iqr)

```
OBSERVACIONES:
Los cambios observados entre los gráficos son suaves, lo que tiene sentido ya que en ambos casos se cortaron los datos extremos usando el IQR, sin embargo se diferencian en que uno reemplaza los valores nulos por el promedio. De esta manera, el único cambio observado es que en las columnas de los promedios para cada caso (alrededor de 3 para los precios y alrededor de 8 para las cantidades) aumentan las cantidades, así permitiendo tener una distribución un poco más robusta y parecida a una normal, aunque no demasiado, ya que el cambio realizado es en un valor super puntual.

```

In [10]:
from sklearn.impute import KNNImputer

numeric_transformations_knn = Pipeline([
    ('remove_outliers', IQR(lambda_)), 
    ('knn_imputer', KNNImputer(n_neighbors=1))
])

column_transformer_knn = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformations_knn, numerical_columns),
        ('categorical', categoric_transformations, categorical_columns)
    ],
    verbose_feature_names_out=False
)

column_transformer_knn.set_output(transform='pandas')

df_knn_imputer = column_transformer_knn.fit_transform(df_retail)

In [ ]:
print("Distribución con imputación usando KNN:")
explore_data(df_knn_imputer)

print("Distribución después de aplicar IQR:")
explore_data(df_iqr)

```
OBSERVACIONES:
Las observaciones son aun más sutiles que antes ya que las cantidades observadas en los histogramas van en la escala de los miles, y a diferencia de antes, ahora la asignación de valores nulos no se concentra en un solo valor, sino que se distribuye en los distintos valores posibles (en particular en los valores más centrales o cercanos a una normal). 

Se observa que en los distintos valores más repetidos la cantidad aumenta un poco, ayudando a tener una distribución más robusta y parecida a una normal, aunque con cambios sutiles. A nivel general hace que los valores importantes que definen la distribución crezcan un poco en repeticiones.


OBSERVACIONES FINALES MEAN_IMPUTER VS KNN_IMPUTER:
Como conclusión de los cambios observados se puede decir que el método de imputación KNN permite obtener resultados más robustos y parecidos a una distribución normal, ya que se a cada dato con un valor faltante le otorga un valor que sea más parecido a los valores que lo rodean según sus otras caracteristicas. Sin embargo, este tiene un costo muy elevado, ya que correrlo con n=1 (que es el valor mínimo) se demoró bastante en correr, mientras que en ese sentido el mean imputer es mucho más eficiente.
```

In [ ]:
df_mean_imputer.isnull().sum()

### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(2 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [13]:
def custom_features(dataframe_in):
    dataframe_in['InvoiceDate'] = pd.to_datetime(dataframe_in['InvoiceDate'])
    
    # Agrupar por Customer ID
    grouped = dataframe_in.groupby('Customer ID').agg(
        length=('InvoiceDate', lambda x: (x.max() - x.min()).days),
        recency=('InvoiceDate', lambda x: (dataframe_in['InvoiceDate'].max() - x.max()).days),
        frequency=('InvoiceDate', 'count'),
        monetary=('Price', 'mean')
    ).reset_index()

    # periodicity 
    df_sorted = dataframe_in.sort_values(['Customer ID', 'InvoiceDate'])
    df_sorted['TimeDiff'] = df_sorted.groupby('Customer ID')['InvoiceDate'].diff().dt.days
    
    periodicity = df_sorted.groupby('Customer ID')['TimeDiff'].std().reset_index()
    periodicity.columns = ['Customer ID', 'Periodicity']

    result = pd.merge(grouped, periodicity, on='Customer ID', how='left')
    
    # Reemplazar NaN en periodicidad con 0, para clientes con una sola compra
    result['Periodicity'].fillna(0, inplace=True)
    
    return result

In [ ]:
df_lrmfp = custom_features(df_retail)
df_lrmfp.head()

#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [15]:
from sklearn.preprocessing import FunctionTransformer

lrmfp_transformer = FunctionTransformer(custom_features)

retail_pipeline = Pipeline([
    ('col_transformer', column_transformer_updated),
    ('custom_features', lrmfp_transformer)    
])

In [ ]:
df_custom = retail_pipeline.fit_transform(df_retail)

In [ ]:
explore_data(df_custom)

```
OBSERVACIONES (2-3 lineas):
Notemos que las distribuciones de length, recency, frequency y de periodicity están mucho más cargadas a valores cercanos a 0. En el caso de monetary se observa una distribución muy parecida a la normal centrada en 2.65. Periodicity fuera de que tiene una gran cantidad de valores 0, también tiene una distribución parecida a la normal centrada en el 8.

INSIGHT DEL NEGOCIO:
La gran mayoría de los valores de Length están concentrados en los valores cercanos a 0, lo que indica que y una baja cantidad de clientes fieles. Esto indica que se requiere mayor cuidado a los clientes y una mejor atención a los nuevos clientes, para así asegurar su mantención a lo largo del tiempo.
```

### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [18]:
class MinMax(BaseEstimator, TransformerMixin):

    def fit(self,X):
        self.max_ = X.max()  #como antes, min y max son string reservados en python por lo que se agrega _
        self.min_ = X.min()
        return self
    def transform(self,X):
        X_scaled = (X-self.min_)/(self.max_ - self.min_)
        return X_scaled

    def set_output(self,transform='default'):
        #No modificar este método
        return self



#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [19]:
numeric_transformations_minmax = Pipeline([
    ('remove_outliers', IQR(lambda_=1.5)), 
    ('mean_imputer', SimpleImputer(strategy='mean')),
    ('minmax', MinMax()) 

])

column_transformer_updated = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformations_minmax, numerical_columns),
        ('categorical', categoric_transformations, categorical_columns)
    ],
    verbose_feature_names_out=False
)
column_transformer_updated.set_output(transform='pandas')

df_minmax = column_transformer_updated.fit_transform(df_retail)

In [ ]:
explore_data(df_retail)

In [ ]:
explore_data(df_minmax)

```
OBSERVACIONES:
Al incluir el scaler MinMax al pipeline que habíamos creado anteriormente para los datos numéricos, se obtiene que los atributos varian entre el 0 y el 1, pero manteniendo las mismas distribuciones que antes, osea, no se alteran las distribuciones de los datos, solo se escala el rango de los datos.
```

### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

```
Los pipelines son muy importantes en el proceso de feature engineering. Estos te permiten evitar la definición y aplicación redundante de muchos pasos, muchas veces, cada vez que tengan que procesar datos nuevos para usarlos en un modelo, o para un análisis.

El uso de los pipelines hace que el proceso sea mucho más simple, que sea mucho más consistente en el tiempo, que el código sea mucho más legible, y que los modelos y los procesos tengan un mejor rendimiento.

Gracias a los pipelines tenemos un flujo mucho más controlado del proceso que se debe realizar desde que llega un dato nuevo, hasta que se usa para entrenar, o para hacer inferencia con un modelo.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>